# Making a Parametric Stellar Population

The core element of the parametric modelling in `Synthesizer` is the `parametric.Stars` object. This contains everything necessary to describe a stellar population including:
- A 2D SFZH grid containg the stellar mass formed in stellar age and metallicity bins.
- A `Morphology` object describing it's distribution (optional but required for imaging).
- The axes of the SFZH grid.
- The totla initial stellar mass.
And if calculated by the user (see the parametric `Sed` [docs](generate_sed.ipynb)):
- The stellar spectra.
- Line luminosities.

There are a number of different methods to define a parametric stellar population (from here on a `Stars` object). In what follows we will focus on user defined values/arrays but note that each different method could instead use simulated data (e.g. from Semi Analytic Models) rather than user defined values.

The possible ways of defining a `Stars` object are (the age and metallicity axis of the SFZH must always be supplied):
- Explictly providing the SFZH grid itself. Note that this will mean all other definitions below are ignored.
- Providing singular age and metallicity values to define an "instantaneous" SFZH, i.e. one where all the stellar mass is in a single age and metallicity bin.
- Providing arrays which contain the SFH and metallicity distribution in terms of stellar mass formed.
- Providing SFH and metallcity distribution functions (from `parametric.sf_hist` and `parametric.metal_hist`). These will be used internal to compute the arrays from the previous point.
- Any combination of the above.

Below we simply import some packages and objects we'll need.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from unyt import yr, Myr
from synthesizer.parametric import SFH, ZH, Stars

## Defining an instantaneous SFZH

To define an instantaneous SFZH we need only to provide the age and metallicity of the SFZH bin, alongside the SFZH axis arrays and the total initial mass of the `Stars` object. Note that the `log10ages` and `log10metallicites`/`metallicities` will likely come from a `Grid` object in most workflows (see the `Grid` [docs](../grids/grids_example.ipynb)).

In [ ]:
# Create the SFZH axis arrays
log10ages = np.arange(6., 10.5, 0.1) # log10(age/yr)
log10metallicities = np.arange(-5., -1.5, 0.25)
metallicities = 10**log10metallicities

# Get the instantaneous Stars
inst_stars = Stars(log10ages, metallicities, instant_sf=100 * Myr, instant_metallicity=10**-3, initial_mass=10 ** 9)

To see the created SFZH you can print some basic statistics by printing the `Stars` object.

In [ ]:
print(inst_stars)

Or to prove the SFZH you can plot it using the provided helper function.

In [ ]:
fig, ax = inst_stars.plot_sfzh(show=True)

## Defining SFZH from arrays

Of course, most of the time you won't want an instantaneous SFZH but instead a distribution. One way to achieve this is to pass a arrays describing the SFH and metallicity distribution. This particularly useful if you have data explictly describing the ages and metallicities of a stellar population. However, here we'll demonstrate this with entirely unphysical arrays.

In [ ]:
# Create arrays for SFH and ZH
sf_hist = np.zeros_like(log10ages)
metal_dist = np.zeros_like(metallicities)
sf_hist[10:15] = 1
sf_hist[15:20] = 0.5
metal_dist[5:6] = 1
metal_dist[6:8] = 0.5

# Get the Stars
arr_stars = Stars(log10ages, metallicities, sf_hist=sf_hist, metal_hist=metal_dist, initial_mass=10 ** 9)
print(arr_stars)
fig, ax = arr_stars.plot_sfzh(show=True)

## Defining SFZH using functions

You won't always have explict arrays to describe the age and metallicity distribtions. For these situations `Synthesizer` provides a suite of functions to describe the SFH and metallicity distribution. 

SFH parametrisations can be found in the `SFH` module, while metallicity distribution parametrisations can be found in the `ZH` module. To see a full list of available parametrisations just print the .

In [ ]:
print(SFH.parametrisations)
print(ZH.parametrisations)

To use one of these parametrisations you have to first define each parametrisation and then pass them to the `Stars` object at instantiation. Below we demonstrate a few examples using different parametrisations.

### Constant SFH and metallicity delta function

In [ ]:
# Define a delta function for metallicity
Z_p = {'log10Z': -2.5}  # can also use linear metallicity e.g. {'Z': 0.01}
Zh = ZH.deltaConstant(Z_p)
print(Zh)

# Define a constant SFH
sfh_p = {'duration': 100 * Myr}
sfh = SFH.Constant(sfh_p) 
print(sfh)

# Create the Stars object
const_stars = Stars(log10ages, metallicities, sf_hist_func=sfh, metal_hist_func=Zh, initial_mass=10**9)
print(const_stars)
fig, ax = const_stars.plot_sfzh(show=True)

### Truncated exponential SFH and metallicity delta function

In [ ]:
# Define a delta function for metallicity
Z_p = {'Z': 0.01}
Zh = ZH.deltaConstant(Z_p)
print(Zh)  # print a summary

# Define an exponential SFH
sfh_p = {'tau': 100 * Myr, 'max_age': 200 * Myr}
sfh = SFH.TruncatedExponential(sfh_p)
print(sfh)  # print summary

# Create the Stars object
exp_stars = Stars(log10ages, metallicities, sf_hist_func=sfh, metal_hist_func=Zh, initial_mass=10**9)
print(exp_stars)
fig, ax = exp_stars.plot_sfzh(show=True)

### Log normal SFH and metallicity delta function

In [ ]:
# Define a delta function for metallicity
Z_p = {'log10Z': -3}
Zh = ZH.deltaConstant(Z_p)
print(Zh)

# Define an exponential SFH
sfh_p = {'tau': 10, 'max_age': 500 * Myr, 'peak_age': 100 * Myr}
sfh = SFH.LogNormal(sfh_p)
print(sfh)

# Create the Stars object
logn_stars = Stars(log10ages, metallicities, sf_hist_func=sfh, metal_hist_func=Zh, initial_mass=10**9)
print(logn_stars)
fig, ax = logn_stars.plot_sfzh(show=True)

### Defining your own parametrisations

If `Synthesizer` doesn't already include a parametrisation you need you can add a custom parametrisation by defining a new class that inherits from the "`Common`" parent class. These custom classes must implment a `sfr_` method for SFHs and `metallicity` and `log10metallicity` methods for metallicity distributions.

Below we demonstrate a custom implementation of (an admittedly pointless) set of "scaled" SFH and metallicity distribution functions for demonstration purposes.

In [ ]:
class ScaledConstantZ(ZH.Common):
    """
    Return a single scaled metallicity as a function of age.

    Attributes:
        name (string)
        dist (string)
        metallicity_ (float)
        log10metallicty_ (float)
        scaling (float)
    """

    def __init__(self, metallicity=None, log10metallicity=None, scaling=1):
        """
        """
        self.name = "Constant"
        self.dist = "delta"  # set distribution type
        self.scaling = scaling
        if metallicity is not None:
            self.metallicity_ = metallicity
            self.log10metallicity_ = np.log10(self.metallicity_)
        else:
            self.log10metallicity_ = log10metallicity
            self.metallicity_ = 10 ** self.log10metallicity_

    def metallicity(self, *args):
        return self.metallicity_ * self.scaling

    def log10metallicity(self, *args):
        return np.log10(10 ** self.log10metallicity_ * scaling)

class ScaledConstantSFH(SFH.Common):
    """
    Return a constant star formation history scaled by a scaling factor.
        sfr = 1; t<=duration
        sfr = 0; t>duration
    """

    def __init__(self, duration, scaling=1):
        self.name = "Constant"
        self.duration = duration.to(yr).value
        self.scaling = scaling

    def sfr_(self, age):
        if age <= self.duration:
            return 1.0 * self.scaling
        else:
            return 0.0

# Define a scaled delta function for metallicity
Zh = ScaledConstantZ(metallicity=0.05, scaling=1)
print(Zh)

# Define an exponential SFH
sfh = ScaledConstantSFH(duration=500 * Myr, scaling=1)
print(sfh)

# Create the Stars object
custom_stars = Stars(log10ages, metallicities, sf_hist_func=sfh, metal_hist_func=Zh, initial_mass=10**9)
print(custom_stars)
fig, ax = custom_stars.plot_sfzh(show=True)



## Combining `Stars`

We can also combine individual `Stars` objects together to produce more complicated star formation and metal enrichment histories by simply adding them.

In [ ]:

combined = arr_stars + const_stars + exp_stars + logn_stars + custom_stars
print(combined)
fig, ax = combined.plot_sfzh()